In [102]:
import sqlite3
import pprint
import pandas as pd
import numpy as np
import re
import plotly
import plotly.graph_objects as go

In [2]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("formula1.sqlite")
cur = con.cursor()

In [3]:
query = """
With R as (SELECT raceId, name, year, name from races),
Re as (SELECT resultId, raceId, driverId, grid, position from results),
D as (Select driverId, surname from drivers)
Select R.name, D.driverid, surname, grid, position, year from Re
INNER JOIN R ON R.raceid == Re.raceid
INNER JOIN D ON D.driverid == Re.driverid
where position <= 3.0
order by year;
"""

In [4]:
df = pd.read_sql_query(query, con)
df

,name,driverId,surname,grid,position,year
0,British Grand Prix,642,Farina,1,1,1950
1,British Grand Prix,786,Fagioli,2,2,1950
2,British Grand Prix,686,Parnell,4,3,1950
3,Monaco Grand Prix,579,Fangio,1,1,1950
4,Monaco Grand Prix,647,Ascari,7,2,1950
...,...,...,...,...,...,...
3070,Brazilian Grand Prix,842,Gasly,6,2,2019
3071,Brazilian Grand Prix,832,Sainz,20,3,2019
3072,Abu Dhabi Grand Prix,1,Hamilton,1,1,2019
3073,Abu Dhabi Grand Prix,830,Verstappen,2,2,2019


In [5]:
query = """
With DS as (SELECT * from driver_standings),
D as (Select driverId, surname from drivers),
R as (SELECT raceId, name, year, name, round from races)
Select R.year, R.round, R.name, D.surname, points from DS
INNER JOIN D ON D.driverId = DS.driverId
INNER JOIN R ON R.raceId = DS.raceId
WHERE YEAR = 2019
ORDER BY R.year ASC, R.round ASC, points DESC;
"""

In [6]:
df = pd.read_sql_query(query, con)
df

,year,round,name,surname,points
0,2019,1,Australian Grand Prix,Bottas,26.0
1,2019,1,Australian Grand Prix,Hamilton,18.0
2,2019,1,Australian Grand Prix,Verstappen,15.0
3,2019,1,Australian Grand Prix,Vettel,12.0
4,2019,1,Australian Grand Prix,Leclerc,10.0
...,...,...,...,...,...
415,2019,21,Abu Dhabi Grand Prix,Magnussen,20.0
416,2019,21,Abu Dhabi Grand Prix,Giovinazzi,14.0
417,2019,21,Abu Dhabi Grand Prix,Grosjean,8.0
418,2019,21,Abu Dhabi Grand Prix,Kubica,1.0


In [7]:
races_rankings = df.groupby('name')

In [9]:
for race in races_rankings:
    print(race)

('Abu Dhabi Grand Prix',      year  round                  name     surname  points
400  2019     21  Abu Dhabi Grand Prix    Hamilton   413.0
401  2019     21  Abu Dhabi Grand Prix      Bottas   326.0
402  2019     21  Abu Dhabi Grand Prix  Verstappen   278.0
403  2019     21  Abu Dhabi Grand Prix     Leclerc   264.0
404  2019     21  Abu Dhabi Grand Prix      Vettel   240.0
405  2019     21  Abu Dhabi Grand Prix       Sainz    96.0
406  2019     21  Abu Dhabi Grand Prix       Gasly    95.0
407  2019     21  Abu Dhabi Grand Prix       Albon    92.0
408  2019     21  Abu Dhabi Grand Prix   Ricciardo    54.0
409  2019     21  Abu Dhabi Grand Prix       Pérez    52.0
410  2019     21  Abu Dhabi Grand Prix      Norris    49.0
411  2019     21  Abu Dhabi Grand Prix   Räikkönen    43.0
412  2019     21  Abu Dhabi Grand Prix  Hülkenberg    37.0
413  2019     21  Abu Dhabi Grand Prix       Kvyat    37.0
414  2019     21  Abu Dhabi Grand Prix      Stroll    21.0
415  2019     21  Abu Dhabi Gra

In [10]:
query = """
With RES as (SELECT * from results),
D as (Select driverId, surname from drivers),
R as (SELECT raceId, name, year, name, round from races)
Select R.year, R.name, R.round, D.surname, grid, position, points, fastestlap, fastestlaptime, rank from RES
INNER JOIN D ON D.driverId = RES.driverId
INNER JOIN R ON R.raceId = RES.raceId
WHERE YEAR = 2019
ORDER BY R.year ASC, R.round ASC, points DESC;
"""

df = pd.read_sql_query(query, con)
df

,year,name,round,surname,grid,position,points,fastestLap,fastestLapTime,rank
0,2019,Australian Grand Prix,1,Bottas,2,1.0,26.0,57.0,1:25.580,1
1,2019,Australian Grand Prix,1,Hamilton,1,2.0,18.0,57.0,1:26.057,2
2,2019,Australian Grand Prix,1,Verstappen,4,3.0,15.0,57.0,1:26.256,3
3,2019,Australian Grand Prix,1,Vettel,3,4.0,12.0,16.0,1:27.954,8
4,2019,Australian Grand Prix,1,Leclerc,5,5.0,10.0,58.0,1:26.926,4
...,...,...,...,...,...,...,...,...,...,...
415,2019,Abu Dhabi Grand Prix,21,Giovinazzi,16,16.0,0.0,28.0,1:43.256,15
416,2019,Abu Dhabi Grand Prix,21,Russell,18,17.0,0.0,50.0,1:43.074,13
417,2019,Abu Dhabi Grand Prix,21,Gasly,11,18.0,0.0,53.0,1:42.414,10
418,2019,Abu Dhabi Grand Prix,21,Kubica,19,19.0,0.0,51.0,1:44.500,20


In [11]:
races = df.groupby('round')

for race in races:
    print(race[1].sort_values('rank'))

    year                   name  round     surname  grid  position  points  \
0   2019  Australian Grand Prix      1      Bottas     2       1.0    26.0   
1   2019  Australian Grand Prix      1    Hamilton     1       2.0    18.0   
2   2019  Australian Grand Prix      1  Verstappen     4       3.0    15.0   
4   2019  Australian Grand Prix      1     Leclerc     5       5.0    10.0   
10  2019  Australian Grand Prix      1       Gasly    17      11.0     0.0   
9   2019  Australian Grand Prix      1       Kvyat    15      10.0     1.0   
8   2019  Australian Grand Prix      1      Stroll    16       9.0     2.0   
3   2019  Australian Grand Prix      1      Vettel     3       4.0    12.0   
5   2019  Australian Grand Prix      1   Magnussen     7       6.0     8.0   
13  2019  Australian Grand Prix      1       Albon    13      14.0     0.0   
7   2019  Australian Grand Prix      1   Räikkönen     9       8.0     4.0   
6   2019  Australian Grand Prix      1  Hülkenberg    11       7

In [12]:
query = """
With L as (SELECT * from lap_times),
D as (Select driverId, surname from drivers),
C as (Select constructorId, color from constructors),
R as (SELECT raceId, name, year, name, round from races)
Select D.surname, C.constructorId, C.color, lap, position, time from L
INNER JOIN D ON D.driverId = L.driverId
INNER JOIN R ON R.raceId = L.raceId
INNER JOIN C ON C.constructorId = L.constructorId
WHERE YEAR = 2019 and R.raceId = '1010'
ORDER BY R.year ASC, R.round ASC;
"""

df = pd.read_sql_query(query, con)

DatabaseError: Execution failed on sql '
With L as (SELECT * from lap_times),
D as (Select driverId, surname from drivers),
C as (Select constructorId, color from constructors),
R as (SELECT raceId, name, year, name, round from races)
Select D.surname, C.constructorId, C.color, lap, position, time from L
INNER JOIN D ON D.driverId = L.driverId
INNER JOIN R ON R.raceId = L.raceId
INNER JOIN C ON C.constructorId = L.constructorId
WHERE YEAR = 2019 and R.raceId = '1010'
ORDER BY R.year ASC, R.round ASC;
': no such column: L.constructorId

In [189]:
def get_results_race(id_race):
    query = """
        With L as (SELECT * from lap_times),
        R as (Select raceId, DriverId, ConstructorId, grid, position, laps from results),
        RA as (SELECT raceId, name, year, name, round from races),
        C as (Select constructorId, name, color from constructors),
        D as (Select driverId, code, surname from drivers)
        Select RA.year, RA.name as GP, C.name, D.surname, D.code as Abb, R.grid, L.lap, L.position, R.position as final, C.color from L
        INNER JOIN RA ON RA.raceId = L.raceId
        INNER JOIN R ON R.driverId = L.driverId and R.raceId = L.raceId
        INNER JOIN D on D.driverId = L.driverId
        INNER JOIN C ON C.ConstructorId = R.ConstructorId 
        WHERE RA.raceId = {}
        """.format(id_race)
    
    df = pd.read_sql_query(query, con)
    gp = str(df['GP'].unique()[0])
#     print(gp)
#     pp = pprint.PrettyPrinter(indent=4)
    
    # Get the best driver of each team
    teams_results = df[['surname', 'name', 'final']].drop_duplicates().groupby('name')
    best_driver_teams = {}
    for team_results in teams_results:
        res = team_results[1]
        tmp = res[res['final'] == res['final'].min()].values
        if len(tmp) > 0:
            best_driver_teams[team_results[0]] = tmp[0]
        else:
            # Both drivers got DNF
            # Setting best long lasting one as best
            bd_dnf = df[df['name'] == team_results[0]][['surname', 'lap']].groupby('surname')['lap'].max().index[0]
            best_driver_teams[team_results[0]] = [bd_dnf]
    
#     pp.pprint(best_driver_teams)
    
    races = df.groupby('surname')
    data = []
    flag = []
    annotations = []
    for race in races:
        dt = race[1].sort_values('lap')
        grid = dt['grid'].iloc[0]
        if grid == 0:
            grid = 20
        color = dt['color'].iloc[0]
#         print(color)
        team = dt['name'].iloc[0]
        name = race[0]
        laps = np.concatenate((np.array([0]), dt['lap'].values))
        pos = dt['position'].values
        positions = np.concatenate((np.array([grid]), pos))
#         print(best_driver_teams[team])
        if color != 'None':
            if name == best_driver_teams[team][0]:
                tmp = go.Scatter(x=laps, y=positions, name=name, mode='lines', line = dict(color=color, width=2))
            else:
                tmp = go.Scatter(x=laps, y=positions, name=name, mode='lines', line = dict(color=color, width=2, dash='dash'))
        else:
            tmp = go.Scatter(x=laps, y=positions, name=name, mode='lines')
            
        data.append(tmp)

        # labeling the right_side of the plot
    #     annotations.append(dict(xref='paper', x=1.05, y=pos[-1],
    #                                   xanchor='auto', yanchor='middle',
    #                                   text='{}'.format(codes[codes['surname']==name]['code'].values[0]),
    #                                   font=dict(family='Arial',
    #                                             size=16),
    #                                   showarrow=False))

    fig = go.Figure(data=data)
    # fig.update_layout(annotations=annotations)
    fig.update_layout(
            title=gp,
            xaxis_title="Laps",
            yaxis_title="Ranking",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )

    fig.show()

In [191]:
for i in range(900,1031):
    get_results_race(i)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [192]:
df

,year,GP,name,surname,Abb,grid,lap,position,final,color
0,2019,British Grand Prix,Mercedes,Hamilton,HAM,2,1,2,1.0,#00D2BE
1,2019,British Grand Prix,Mercedes,Hamilton,HAM,2,2,2,1.0,#00D2BE
2,2019,British Grand Prix,Mercedes,Hamilton,HAM,2,3,2,1.0,#00D2BE
3,2019,British Grand Prix,Mercedes,Hamilton,HAM,2,4,2,1.0,#00D2BE
4,2019,British Grand Prix,Mercedes,Hamilton,HAM,2,5,2,1.0,#00D2BE
...,...,...,...,...,...,...,...,...,...,...
908,2019,British Grand Prix,Haas F1 Team,Magnussen,MAG,16,2,20,NaN,#F0D787
909,2019,British Grand Prix,Haas F1 Team,Magnussen,MAG,16,3,20,NaN,#F0D787
910,2019,British Grand Prix,Haas F1 Team,Magnussen,MAG,16,4,20,NaN,#F0D787
911,2019,British Grand Prix,Haas F1 Team,Magnussen,MAG,16,5,20,NaN,#F0D787
